# Processing Hazelnut Yield

The raw data provided contains data points with timestamps, GPS coordinates, and collection cart weight. This information was collected as a cart moves along rows of trees and harvests nuts.

The goal is to create a heatmap of productivity. We measure change in weight with respect to distance traveled, and pool data from multiple harvests to display the total productivity spatially.

In [1]:
import sys
import re
import os
import json
import pandas as pd
import geopandas
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import webcolors

Our method relies on configuring the boundaries of the fields. In the [configuration file](files/config.json), the coordinates of the corners of multiple fields are specified.

In [2]:
def create_field_polygons(cfg):
    # Creating a GeoPandas polygon for each field in config
    field_corners = {}
    for field in cfg['field_boundaries']:
        field_corners[field['name']] = Polygon(
                            [(field['NW_corner']['latitude'],field['NW_corner']['longitude']),
                            (field['NE_corner']['latitude'], field['NE_corner']['longitude']),
                            (field['SE_corner']['latitude'], field['SE_corner']['longitude']),
                            (field['SW_corner']['latitude'], field['SW_corner']['longitude'])])
    field_polygons = geopandas.GeoSeries(field_corners)
    return field_polygons

When we load the raw CSV lists, we ignore null points:

In [3]:
def load_file(cfg, filename):
    csv_directory = cfg['csv_directory']
    df = pd.read_csv(f'{csv_directory}/{filename}', encoding = "ISO-8859-1")

    data = []
    for index, row in df.iterrows():
        try:
            point = Point(float(row['lat']), float(row['long']))
            # If the point does not contain zeros, add to list
            if not np.isnan(point.x) and not np.isnan(point.y):
                data.append({'latitude': row['lat'], 'longitude':row['long'], 'weight':row['weight']})
        except:
            pass
    return data

The quantize_yield_intensity function does the heavy lifting. It iterates over points until the configured distance interval is exceeded. It then calculates change in weight over distance (yield intensity), and applies that property to a rectangle surrounding the path traversed. This removes the necessity of a fixed grid overlayed on the field, allowing for flexibility in resolution. The preciseness is limited by either the configured distance interval, or the frequecy of data collection.

The rectangles are trimmed by the field boundary, so the configured corners should be strict so travel between rows is removed.

Helper functions exist to create these rectangles, and to remove noisy data. Weight shifting around the cart is identified by negative weight changes. We also remove preceeding and succeeding data points to negative weight changes, as these points have artifically high weight changes due to the nuts shifting back.

In [4]:
def quantize_yield_intensity(cfg, data, field_polygons):
    yield_intensities = {}
    for field in cfg['field_boundaries']:
        field_name = field['name']
        yield_intensities[field_name] = []

        if len(data) > 0:
            initial_point = data[0]
            for i in range(0, len(data)):
                initial_location = Point(float(initial_point['latitude']), float(initial_point['longitude']))
                i_location = Point(float(data[i]['latitude']), float(data[i]['longitude']))
                distance = geopandas.GeoSeries(initial_location).distance(geopandas.GeoSeries(i_location))[0]
                try:
                    # Removing anything that isn't a decimal point or a digit from recorded weights
                    i_weight = float(re.sub('[^A-Za-z0-9.]+', '', data[i]['weight']))
                    initial_weight = float(re.sub('[^A-Za-z0-9.]+', '', initial_point['weight']))

                    yield_intensity =  {}
                    yield_intensity['intensity'] = (i_weight - initial_weight)/distance
                    yield_intensity['line'] = create_line(initial_location, i_location, field_name)
                    yield_intensities[field_name].append(yield_intensity)
                        
                    # Reset initial point for next iteration
                    initial_point = data[i]
                except:
                    pass
                    

    return yield_intensities

This function pools the data from each harvest day. Intensities of intersecting areas are summed. Some smoothing needs to be implemented to avoid small polygons with outlier intensities.

The following block uses the defined functions sequentially. It will take roughly 10 minutes.

The following block exports the data for visualization on geojson.io

In [ ]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b

for field in cfg['field_boundaries']:
    field_name = field['name']
    export_list = []
    max_intensity = 0
    for entry in total_yield_intensities[field_name]:
        if entry['yield_intensity'] > max_intensity:
            max_intensity = entry['yield_intensity']

    for entry in total_yield_intensities[field_name]:
        entry['region']['yield_intensity'] = entry['yield_intensity']
        entry['region']['fill'] = webcolors.rgb_to_hex(rgb(0, max_intensity, entry['yield_intensity']))
        export_list.append(geopandas.GeoDataFrame(entry['region']))

    export_object = geopandas.GeoDataFrame(pd.concat(export_list, ignore_index=True))
    export_object.to_file(f'{field_name}_intensity.json', driver="GeoJSON")

    # geojson.io requires coordinates in lat, long
    with open(f'{field_name}_intensity.json', 'r') as json_data_file:
        yield_intensities = json.load(json_data_file)
        for feature in yield_intensities['features']:
            polygon = feature['geometry']['coordinates']
            for corner in polygon:
                for coordinates in corner:
                    long = coordinates[0]
                    lat = coordinates[1]
                    coordinates[0] = lat
                    coordinates[1] = long

        with open(f'{field_name}_intensity_lat_long.json', 'w') as outfile:
            json.dump(yield_intensities, outfile)